In [ ]:
import pandas as pd

from src.data_processing.wrangling import cat_conditions, map_comp, map_investimento, map_importo

In [ ]:
p01 = pd.read_parquet("data/updated/p01.parquet")
p02 = pd.read_parquet("data/updated/p02.parquet")
p03 = pd.read_parquet("data/updated/p03.parquet")
p04 = pd.read_parquet("data/updated/p04.parquet")
p05 = pd.read_parquet("data/updated/p05.parquet")

componenti = pd.read_parquet("data/updated/components.parquet")
missions = pd.read_parquet("data/updated/missions.parquet")
comuni = pd.read_parquet("data/updated/comuni.parquet")

## Data preprocessing

In [ ]:
p01.shape

In [ ]:
p01.columns

In [ ]:
p02.columns

In [ ]:
p02.shape

In [ ]:
p02.head()

In [ ]:
p03 = p03.rename(columns={'cig':'CIG'})
p03['CATEGORIA'] = p03.apply(cat_conditions,axis=1)

In [ ]:
print(p03.shape)
print(p03.columns)
p03.head()

In [ ]:
p04 = p04.rename(columns={'cig':'CIG'})
print(p04.shape)
print(p04.columns)
p04.head()

In [ ]:
p05['importo_complessivo_gara'].unique()

In [ ]:
p05 = p05.rename(columns={'cig':'CIG'})
p05['importo_complessivo_gara'] = p05['importo_complessivo_gara'].astype(float)
p05['CLASSE_IMPORTO'] = p05['importo_complessivo_gara'].apply(map_importo)

In [ ]:
print(p05.shape)
print(p05.columns)
p05.head()

## Merge dei Dataset "P0"

In [ ]:
# missioni e componenti, left join da p02
p02 = p02.merge(missions,left_on="Missione", right_on=missions.index, how='inner')
p02 = p02.drop_duplicates()
p02 = p02.merge(componenti,left_on="Componente", right_on=componenti.index, how='inner')
p02 = p02.drop_duplicates()

In [ ]:
comuni.columns

In [ ]:
p05.columns

In [ ]:
p05['luogo_istat']

In [ ]:
comuni['Codice Comune Alfanumerico'] = comuni['Codice Comune Alfanumerico'].astype(str)
comuni['Codice Comune Alfanumerico']

In [ ]:
# left join p05 con i dati su comuni e denominazioni 
p05=p05.merge(
    comuni,
    left_on='luogo_istat', 
    right_on='Codice Comune Alfanumerico',
    how='left'
)
p05=p05.drop_duplicates()

In [ ]:
# left join p5 e p3
leftnew_3 = p05.merge(p03, on='CIG', how='left')
leftnew_3 = leftnew_3.drop_duplicates()
print(leftnew_3.shape)

In [ ]:
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG', how='left')
leftnew_34 = leftnew_34.drop_duplicates()
print(leftnew_34.shape)

In [ ]:
# left join il risultato con p1
leftnew_341 = leftnew_34.merge(p01, on='CIG', how='left')
leftnew_341 = leftnew_341.drop_duplicates()
print(leftnew_341.shape)

In [ ]:
# left join il risultato con p2
leftnew_3412 = leftnew_341.merge(p02,how='left',left_on='CUP',right_on='CUP')
leftnew_3412 = leftnew_3412.drop_duplicates()
print(leftnew_3412.shape)

In [ ]:
colonne_da_selezionare = [
    'CIG', 
    'CUP',  
    'Regione', 
    'provincia',  
    'Comune', 
    'importo_complessivo_gara', 
    'CLASSE_IMPORTO', 
    'Descrizione Missione', 
    'Missione', 
    'Componente',  
    'cod_mis_premiale', 
    'misura_premiale', 
    'CATEGORIA', 
    'flag_quote', 
    'quota_femminile', 
    'quota_giovanile', 
    'cod_mot_deroga', 
    'mot_deroga', 
    'data_pubblicazione', 
    'anno_pubblicazione', 
    'FLAG_URGENZA', 
    'MOTIVO_URGENZA', 
    'ESITO',
    'Codice Univoco Submisura'
]

In [ ]:
master_table = leftnew_3412[colonne_da_selezionare]

In [ ]:
master_table['Comune'].nunique()

In [ ]:
master_table.shape

In [ ]:
master_table.head(5)

In [ ]:
# master_table.to_csv("data/updated/master_table_base.csv", sep=";")

# Merge Dati ItaliaDomani

In [1]:
import pandas as pd
master_table = pd.read_csv("data/updated/master_table_base.csv", sep=";")
indicatori_sdg = pd.read_parquet("data/updated/indicators_sdg.parquet")
indicators = pd.read_parquet("data/updated/indicators.parquet")
targets = pd.read_parquet("data/updated/targets.parquet")
indicatori_com_regis = pd.read_parquet("data/updated/universo_regis.parquet")

### Indicators

In [2]:
master_table.columns

Index(['Unnamed: 0', 'CIG', 'CUP', 'Regione', 'provincia', 'Comune',
       'importo_complessivo_gara', 'CLASSE_IMPORTO', 'Descrizione Missione',
       'Missione', 'Componente', 'cod_mis_premiale', 'misura_premiale',
       'CATEGORIA', 'flag_quote', 'quota_femminile', 'quota_giovanile',
       'cod_mot_deroga', 'mot_deroga', 'data_pubblicazione',
       'anno_pubblicazione', 'FLAG_URGENZA', 'MOTIVO_URGENZA', 'ESITO',
       'Codice Univoco Submisura'],
      dtype='object')

In [3]:
# for col in leftnew_3412.columns:
#     print(col)

In [4]:
# andiamo in merge su Codice Univoco Submisura
indicators.columns 

Index(['Programma', 'Missione', 'Descrizione Missione', 'Componente',
       'Descrizione Componente', 'ID Misura', 'Codice Univoco Misura',
       'Descrizione Misura', 'ID Submisura', 'Codice Univoco Submisura',
       'Codice CID', 'Descrizione Submisura', 'Amministrazione Titolare',
       'Codice Indicatore', 'Descrizione Indicatore', 'Unità di misura',
       'Stock/Flusso'],
      dtype='object')

In [5]:
indicators['Codice Univoco Submisura'].isna().sum()

0

In [6]:
indicators.head()

,Programma,Missione,Descrizione Missione,Componente,Descrizione Componente,ID Misura,Codice Univoco Misura,Descrizione Misura,ID Submisura,Codice Univoco Submisura,Codice CID,Descrizione Submisura,Amministrazione Titolare,Codice Indicatore,Descrizione Indicatore,Unità di misura,Stock/Flusso
0,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.3,M1C1I1.03,Dati e interoperabilità,M1C1I1.3.2,M1C1I1.03.02,M1C1I1.3.2,Sportello digitale unico,PCM - DIPARTIM. TRASFORMAZIONE DIGITALE,C7,"UTENTI DI SERVIZI, PRODOTTI E PROCESSI DIGITAL...",Utenti / anno,Flusso
1,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.4,M1C1I1.04,Servizi digitali e esperienza dei cittadini,M1C1I1.4.3,M1C1I1.04.03,M1C1I1.4.3,Rafforzamento dell'adozione dei servizi della ...,PCM - DIPARTIM. TRASFORMAZIONE DIGITALE,C7,"UTENTI DI SERVIZI, PRODOTTI E PROCESSI DIGITAL...",Utenti / anno,Flusso
2,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.4,M1C1I1.04,Servizi digitali e esperienza dei cittadini,M1C1I1.4.4,M1C1I1.04.04,M1C1I1.4.4,Rafforzamento dell'adozione delle piattaforme ...,PCM - DIPARTIM. TRASFORMAZIONE DIGITALE,C7,"UTENTI DI SERVIZI, PRODOTTI E PROCESSI DIGITAL...",Utenti / anno,Flusso
3,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.4,M1C1I1.04,Servizi digitali e esperienza dei cittadini,M1C1I1.4.5,M1C1I1.04.05,M1C1I1.4.5,Digitalizzazione degli avvisi pubblici,PCM - DIPARTIM. TRASFORMAZIONE DIGITALE,C7,"UTENTI DI SERVIZI, PRODOTTI E PROCESSI DIGITAL...",Utenti / anno,Flusso
4,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.4,M1C1I1.04,Servizi digitali e esperienza dei cittadini,M1C1I1.4.6,M1C1I1.04.06,M1C1I1.4.6,Mobilità come servizio per l'Italia,PCM - DIPARTIM. TRASFORMAZIONE DIGITALE,C7,"UTENTI DI SERVIZI, PRODOTTI E PROCESSI DIGITAL...",Utenti / anno,Flusso


In [7]:
master_table['Codice Univoco Submisura'].isna().sum()

63713

In [8]:
master_table_ID = master_table.merge(
    indicators,
    how='left',
    left_on='Codice Univoco Submisura',
    right_on='Codice Univoco Submisura'
)
master_table_ID = master_table_ID.drop_duplicates()
print(master_table_ID.shape)

(594101, 41)


## Indicatori SDG

In [9]:
indicatori_sdg.shape

(553, 26)

In [10]:
# qua andiamo in merge con COD_SUB_MISURA
indicatori_sdg.columns 

Index(['COD_MISSIONE', 'DES_MISSIONE', 'COD_COMPONENTE', 'DES_COMPONENTE',
       'COD_MISURA', 'DES_MISURA', 'COD_SUB_MISURA', 'DES_SUB_MISURA',
       'DES_LIVELLO', 'IMPORTO', 'DES_INVESTIMENTO', 'DES_PRESTITO',
       'DES_AMMINISTRAZIONE', 'PREVALENTE', 'INDICATORE', 'DES_INDICATORE',
       'DES_INDAGINE', 'NOTA', 'DES_FONTE', 'DES_UNITA', 'GOAL', 'DES_GOAL',
       'TARGET', 'DES_TARGET', 'INDICATOR', 'DES_INDICATOR'],
      dtype='object')

In [11]:
indicatori_sdg['COD_SUB_MISURA'].isna().sum()

0

In [12]:
indicatori_sdg.head()

,COD_MISSIONE,DES_MISSIONE,COD_COMPONENTE,DES_COMPONENTE,COD_MISURA,DES_MISURA,COD_SUB_MISURA,DES_SUB_MISURA,DES_LIVELLO,IMPORTO,...,DES_INDAGINE,NOTA,DES_FONTE,DES_UNITA,GOAL,DES_GOAL,TARGET,DES_TARGET,INDICATOR,DES_INDICATOR
0,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.01,Infrastrutture digitali,M1C1I1.01.00,Infrastrutture digitali,Misura,"900000000,00",...,None,None,None,None,16,Promuovere società pacifiche e inclusive per u...,None,None,None,None
1,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.02,Abilitazione al cloud per le PA locali,M1C1I1.02.00,Abilitazione al cloud per le PA locali,Misura,"1000000000,00",...,None,None,None,None,16,Promuovere società pacifiche e inclusive per u...,None,None,None,None
2,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.03,Dati e interoperabilità,M1C1I1.03.01,Piattaforma Digitale Nazionale Dati,Sub-Misura,"556000000,00",...,None,None,None,None,16,Promuovere società pacifiche e inclusive per u...,None,None,None,None
3,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.03,Dati e interoperabilità,M1C1I1.03.02,Sportello digitale unico,Sub-Misura,"90000000,00",...,None,None,None,None,16,Promuovere società pacifiche e inclusive per u...,None,None,None,None
4,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.04,Servizi digitali e esperienza dei cittadini,M1C1I1.04.01,Esperienza dei cittadini - Miglioramento della...,Sub-Misura,"613000000,00",...,None,None,None,None,16,Promuovere società pacifiche e inclusive per u...,None,None,None,None


In [13]:
master_table_ID = master_table_ID.merge(
    indicatori_sdg,
    how='left',
    left_on='Codice Univoco Submisura',
    right_on='COD_SUB_MISURA'
)
master_table_ID = master_table_ID.drop_duplicates()
print(master_table_ID.shape)

(918923, 67)


## Targets

In [14]:
targets.shape

(84893, 13)

In [15]:
# scelta tra CUP e Codice Univoco Submisura. CUP Preferibile 
targets.columns

Index(['Codice Univoco Submisura', 'Descrizione Submisura', 'CUP',
       'Codice Locale Progetto', 'Codice Indicatore', 'Descrizione Indicatore',
       'Unità di Misura', 'Descrizione Unità di Misura', 'Valore Programmato',
       'Valore Realizzato', 'Mese', 'Anno', 'Data di Aggiornamento'],
      dtype='object')

In [16]:
targets['Codice Univoco Submisura'].isna().sum()

0

In [17]:
targets['CUP'].isna().sum()

0

In [18]:
targets['CUP'].nunique()

50539

In [19]:
targets['Codice Univoco Submisura'].nunique()

103

In [20]:
targets.head()

,Codice Univoco Submisura,Descrizione Submisura,CUP,Codice Locale Progetto,Codice Indicatore,Descrizione Indicatore,Unità di Misura,Descrizione Unità di Misura,Valore Programmato,Valore Realizzato,Mese,Anno,Data di Aggiornamento
0,M1C1I1.01.00,Infrastrutture digitali,D31C23000790006,PRJ_110ASL0323X_000100,T0081,NR. DI AMMINISTRAZIONI MIGRATE AL POLO STRATEG...,12,Numero,"0,00","0,00",Settembre,2023,20/02/2024
1,M1C1I1.01.00,Infrastrutture digitali,F71C23000580006,PRJ_110ASL0323X_000090,T0081,NR. DI AMMINISTRAZIONI MIGRATE AL POLO STRATEG...,12,Numero,"0,00","0,00",Settembre,2023,20/02/2024
2,M1C1I1.01.00,Infrastrutture digitali,G21C23000500006,PRJ_110ASL0323X_000146,T0081,NR. DI AMMINISTRAZIONI MIGRATE AL POLO STRATEG...,12,Numero,"0,00","0,00",Settembre,2023,20/02/2024
3,M1C1I1.01.00,Infrastrutture digitali,F31C23000610006,PRJ_110ASL0323X_000089,T0081,NR. DI AMMINISTRAZIONI MIGRATE AL POLO STRATEG...,12,Numero,"0,00","0,00",Settembre,2023,20/02/2024
4,M1C1I1.01.00,Infrastrutture digitali,J81C23000680006,PRJ_110ASL0323X_000118,T0081,NR. DI AMMINISTRAZIONI MIGRATE AL POLO STRATEG...,12,Numero,"0,00","0,00",Settembre,2023,20/02/2024


: 

In [21]:
master_table_ID = master_table_ID.merge(
    targets,
    how='left',
    left_on='CUP',
    right_on='CUP'
)
master_table_ID = master_table_ID.drop_duplicates()
print(master_table_ID.shape)

### Universo Regis

In [ ]:
indicatori_com_regis.shape

In [ ]:
# Scegliamo il CUO
indicatori_com_regis.columns

In [ ]:
indicatori_com_regis['CUP'].isna().sum()

In [ ]:
indicatori_com_regis['Codice Univoco Submisura'].isna().sum()

In [ ]:
master_table_ID = master_table_ID.merge(
    indicatori_com_regis,
    how='left',
    left_on='CUP',
    right_on='CUP'
)
master_table_ID = master_table_ID.drop_duplicates()
print(master_table_ID.shape)

In [ ]:
# indicators

# indicators sdg

# targets

# universo_regis